In [1]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
# a_pattern = 'SI06322.Chn'
# a_pattern = 'SI06122.Chn'

In [2]:
import numpy as np
from numpy.polynomial import Polynomial as P  # 2020-09-06 Esta é a nova classe recomendada
from sklearn import linear_model

import pandas as pd
from random import randrange
# import plotly.graph_objects as go

In [3]:
from ograyspy_class import Ograyspy, select_spectrum_from_folder_list
from spec_class import Spec

In [4]:
ogra = Ograyspy()
print(ogra.info_node)
to_be_found = 'Genie_Transfer'
# to_be_found = 'some_spectra' # NAO ESTAH ACHANDO localmente no note!

Found folder name:  C:\Users\mmaduar\.atom
Parts:  ('C:\\', 'Users', 'mmaduar', '.atom')
No. spec files:  0
I56611


In [5]:
print('\nExec ogra.define_files_folder(to_be_found):')
ogra.define_files_folder(to_be_found)
print(ogra.pkl_folder_files)
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
del ogra


Exec ogra.define_files_folder(to_be_found):
Found folder name:  C:\Users\mmaduar\Nextcloud\Genie_Transfer
Parts:  ('C:\\', 'Users', 'mmaduar', 'Nextcloud', 'Genie_Transfer')
No. spec files:  7316
WindowsI56611.pkl


In [6]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [7]:
len(reduc_nms)

7316

In [8]:
# 2022-Dez-28 Esse é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06122.Chn'

In [9]:
f_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)
f_name, reduced_f_name

Existing:
Achou! indice=7212, nomearq = Si/SI2022/SI06122.Chn
Final choices:
spectra_path: C:\Users\mmaduar\Nextcloud\Genie_Transfer
a_spec_name: C:\Users\mmaduar\Nextcloud\Genie_Transfer\Si\SI2022\SI06122.Chn
reduced_f_name: Si/SI2022/SI06122.Chn


(WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/Si/SI2022/SI06122.Chn'),
 'Si/SI2022/SI06122.Chn')

In [10]:
nucl_iear1_df = pd.read_pickle('nucl_iear1_list.pkl')
nucl_iear1_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma
1,24na,1368.6250,0.005,99.994,0.002,14.956,B-,100.0,1368.6,True
2,24na,2754.0080,0.011,99.867,0.01,14.956,B-,100.0,1368.6,False
9,51cr,320.0824,0.0004,9.910,0.01,27.704,EC,100.0,320.1,True
15,54mn,834.8480,0.003,99.976,0.001,312.200,EC+B+,100.0,834.8,True
22,58co,810.7593,0.002,99.450,0.01,70.860,EC+B+,100.0,810.8,True
...,...,...,...,...,...,...,...,...,...,...
1066,228Ac,89.9540,,2.010,0.19,6.150,B-,100.0,911.6,False
1067,228Ac,93.3470,,3.200,0.3,6.150,B-,100.0,911.6,False
1068,228Ac,105.5660,,1.160,0.11,6.150,B-,100.0,911.6,False
1069,228Ac,106.8940,,1.540,0.15,6.150,B-,100.0,911.6,False


In [11]:
# 2023-Jun-15: Setting gamma lines/ranges to dismiss in the analysis
df1 = nucl_iear1_df
df1["is_to_consider"] = True
df1.loc[((df1.energy > 509) & (df1.energy < 513)) | (df1.energy < 100), "is_to_consider"] = False
# Convert these columns from object to numeric; empty strings turns into NaNs
df1["unc_i"] = pd.to_numeric(df1.unc_i, errors="coerce")
df1["unc_en"] = pd.to_numeric(df1.unc_en, errors="coerce")
del df1
nucl_iear1_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider
1,24na,1368.6250,0.0050,99.994,0.002,14.956,B-,100.0,1368.6,True,True
2,24na,2754.0080,0.0110,99.867,0.010,14.956,B-,100.0,1368.6,False,True
9,51cr,320.0824,0.0004,9.910,0.010,27.704,EC,100.0,320.1,True,True
15,54mn,834.8480,0.0030,99.976,0.001,312.200,EC+B+,100.0,834.8,True,True
22,58co,810.7593,0.0020,99.450,0.010,70.860,EC+B+,100.0,810.8,True,True
...,...,...,...,...,...,...,...,...,...,...,...
1066,228Ac,89.9540,NaN,2.010,0.190,6.150,B-,100.0,911.6,False,False
1067,228Ac,93.3470,NaN,3.200,0.300,6.150,B-,100.0,911.6,False,False
1068,228Ac,105.5660,NaN,1.160,0.110,6.150,B-,100.0,911.6,False,True
1069,228Ac,106.8940,NaN,1.540,0.150,6.150,B-,100.0,911.6,False,True


In [12]:
a_spec = Spec(f_name, reduced_f_name)

In [38]:
vars (a_spec)

{'net_spec_ser_an': None,
 'final_composed_baseline': None,
 'f_name': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.Chn'),
 'reduced_f_name': 'almera22/user2023/SI2022/SI06322.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.58,
 'source_datetime': None,
 'results_pkl_file': WindowsPath('.'),
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x1d74630b190>,
 'start_datetime': datetime.datetime(2022, 8, 4, 19, 42, 23),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-063/22 - Agua da Piscina do Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x1d74630b3d0>,
 'nucl_an': <nuclide_analysis_class.NuclideAnalysis at 0x1d74630b5e0>}

In [13]:
parar aqio

SyntaxError: invalid syntax (2640695910.py, line 1)

In [ ]:
dir(a_spec)

In [ ]:
a_spec.total_analysis(gener_dataframe=True, results_path='../ograyspy_results')
# A opção acima, gener_dataframe=True,  grava o pkl_file do espectro
vars(a_spec)

In [ ]:
a_spec.identify_nuclides(nucl_iear1_df)

In [ ]:
aux_3, pks_comprehensive_df, df_result, d5_grouped, res_1 = (
    a_spec.create_activities_df(nucl_iear1_df, results_path='../ograyspy_results')
)

In [ ]:
aux_3

In [ ]:
pks_comprehensive_df

In [ ]:
df_result

In [ ]:
df_result.columns

In [ ]:
df_result[['peaks','rough_sums','nuclide_name']]

In [ ]:
d5_grouped

In [ ]:
res_1

In [ ]:
# 2023-Oct-25 PAREI AQUI. Reformatar-deletar tudo o que segue.

In [ ]:
justtostopthenotebook

In [ ]:
dir(a_spec)

In [ ]:
spectra_list_df

In [ ]:
nucl_iear1_selctd_gamms_df = nucl_iear1_df.loc [
    (nucl_iear1_df.intensity > 1.0) & nucl_iear1_df.is_to_consider
]
nucl_iear1_selctd_gamms_df

In [ ]:
nucl_iear1_selctd_gamms_df.dtypes

In [ ]:
# Primeiro fiz assim:
# cross_df = pd.merge(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, how='cross')
# cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
# cross_df

# Agora fazer de novo:
# cross_df_2 = pd.merge(matching_peaks_df, nucl_iear1_selctd_gamms_df, how='cross', suffixes=('_x', None))
# cross_df_2.drop(cross_df_2.filter(regex='_x$').columns, axis=1, inplace=True)
# cross_df_2["delta_en"] = cross_df_2.recalib_engy_ransac - cross_df_2.energy
# cross_df_2

In [ ]:
# cross_df = pd.merge(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, how='cross')
 #    cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
  #  cross_df 

In [ ]:
# 2023-Oct-25
# Proceed from here.

In [ ]:
new_ch_en_calib

In [ ]:
def en_eff(engy):
    return np.exp(p_eff(np.log(engy)))

In [ ]:
en_eff(1332)

In [ ]:
# 2023-Sep-21
# Now use the assembled peak df, read_result_df, to proceed.

In [14]:
# 2023-Oct-24: FIX
# Integrate what follows here

In [15]:
analysis_df.reset_index(drop=True, inplace=True)

NameError: name 'analysis_df' is not defined

In [ ]:
analysis_df['test'] = analysis_df['decay_%']*analysis_df['intensity']/100

In [ ]:
analysis_df

In [ ]:
analysis_df.reindex(['nuclide_name'])

In [ ]:
# d5_where = d5.where(d5.nuclide_name=="110ag")
# d5_where
d5_loc = analysis_df.loc[analysis_df.nuclide_name=="110ag"]
d5_loc

In [ ]:
# 2023-Ago-4 PAREI AQUI. Fazer a indexação para o cálculo da atividade considerando a média das linhas de cada nuclídeo
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#dataframe-column-selection-in-groupby
d5_grouped = analysis_df.groupby("nuclide_name")[['energy','intensity','centroids','rough_sums','variances','disintegr']]
# d5_grouped = d5.groupby("nuclide_name")[['disintegr']]
# sdfsdf = pd.DataFrame([[d5_grouped.mean(), d5_grouped.median()]]) 
# sdfsdf


In [ ]:
d5_grouped.mean()

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#built-in-aggregation-methods
d5_aggr_df = pd.DataFrame(df_result.groupby("nuclide_name")[['disintegr']].mean())
d5_aggr_df['dsnt_median'] = df_result.groupby("nuclide_name")[['disintegr']].median()
d5_aggr_df['dsnt_counts'] = df_result.groupby("nuclide_name")[['disintegr']].count()
d5_aggr_df['dsnt_sem'] = df_result.groupby("nuclide_name")[['disintegr']].sem()
d5_aggr_df['dsnt_std'] = df_result.groupby("nuclide_name")[['disintegr']].std()
d5_aggr_df

In [ ]:
df_result.groupby("nuclide_name")[['disintegr']].describe()

In [ ]:
d5_aggr_df['disint_mean'] = d5_grouped.mean()
d5_aggr_df['disint_count'] = d5_grouped.count()
d5_aggr_df

In [ ]:
df_result

In [ ]:
help(spec_engy_identif)

In [ ]:
# 2023-Jun-13
# PAREI AQUI
# Trazer o trabalho feito em
# radionuclide_analysis.ipynb e adaptar ao que está aqui
# especialmente na identificaćão dos nucl[ideos

# Depois, deletar radionuclide_analysis.ipynb.

In [ ]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)


df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)


df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)


frames = [df1, df2, df3]

result = pd.concat(frames)
result

In [ ]:
result = pd.concat(frames, keys=["x", "y", "z"])
result

In [ ]:
result.loc[result.A=='A1'].loc[result.C=='C0']

In [ ]:
# https://stackoverflow.com/questions/63529555

In [ ]:
animals = ['cat', 'dog', 'hamster', 'dolphin']

In [ ]:
df_animals_sets = pd.DataFrame(data={
    'id': [1,2,3,4,5],
    'animals': ['dog,cat','dog','cat,dolphin','cat,dog','hamster,dolphin']
})
df_animals_sets

In [ ]:
df = df_animals_sets
df_final = (df.astype(str).assign(animals=df.animals.str.split(','))
                          .explode('animals').groupby('animals').id.agg(','.join)
                          .reset_index())
df_final

In [16]:
# 2023-Oct-24 FIX:
# Brought from extinct radionuculde_analysis.ipynb
# Jupyter notebook for tests with OGRaySpY modules

In [17]:
pip list

Package                       Version
----------------------------- ------------
alabaster                     0.7.13
anyio                         4.0.0
argon2-cffi                   23.1.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
asteval                       0.9.31
asttokens                     2.4.0
async-lru                     2.0.4
attrs                         23.1.0
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.12.2
bleach                        6.0.0
certifi                       2023.7.22
cffi                          1.15.1
charset-normalizer            3.2.0
colorama                      0.4.6
comm                          0.1.4
contourpy                     1.1.1
cycler                        0.11.0
debugpy                       1.8.0
decorator                     5.1.1
defusedxml                    0.7.1
docutils                      0.20.1
exceptiongroup                1.1.3
execu

You should consider upgrading via the 'C:\Users\mmaduar\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [18]:
import numpy as np
from numpy.random import default_rng
from numpy.polynomial import Polynomial as P
from scipy import special
import pandas as pd

from ograyspy_class import select_spectrum_from_folder_list
from spec_class import Spec

In [19]:
# https://lmfit.github.io/lmfit-py/builtin_models.html
from lmfit import Model, minimize, Parameters, report_fit, fit_report, printfuncs
from lmfit.models import LinearModel, StepModel, ExponentialModel, LorentzianModel
from lmfit.models import GaussianModel, SkewedGaussianModel, ExpressionModel

In [20]:
from ograyspy_class import Ograyspy

In [21]:
ogra = Ograyspy(batch_mode=False, folder_to_find='Genie_Transfer')
print('Objeto ogra: Ograyspy criado.')
print(f'ogra.info_plat: {ogra.info_plat}')
print(f'ogra.info_mach: {ogra.info_mach}')
print(f'ogra.info_syst: {ogra.info_syst}')
print(f'ogra.info_node: {ogra.info_node}')
print(f'ogra.home_path: {ogra.home_path}')

Found folder name:  C:\Users\mmaduar\Nextcloud\Genie_Transfer
Parts:  ('C:\\', 'Users', 'mmaduar', 'Nextcloud', 'Genie_Transfer')
No. spec files:  7316
Objeto ogra: Ograyspy criado.
ogra.info_plat: Windows-10-10.0.19045-SP0
ogra.info_mach: AMD64
ogra.info_syst: Windows
ogra.info_node: I56611
ogra.home_path: C:\Users\mmaduar


In [22]:
# a spectrum pattern to find and test:

In [23]:
# 2022
# Se quiser ignorar o espectro aleatório, definir aqui um específico. Se não, pular.
# Vou ignorar o espectro já analisado e refazer a análise, para testes.
# 2022-Mar-24 Resolver essa questão dos nomes de arquivos em Linux e Windows
# a_spec_name = '..\\..\\Genie_Transfer\\Si\\2018\\PNR\\PRN0803.Chn'
# a_spec_name = '../../Genie_Transfer/Si/2018/PNR/PRN0803.Chn'
# a_spec_name = '../../Genie_Transfer/Si/2018/CTP/CTP0908-2.Chn'
# Esse é um que dá problema na baseline perto do 159 keV do I-123:
# a_spec_name = '../../Genie_Transfer/Si/2018/CCI/CCI1508I.Chn'
# Precisa quebrar mais a baseline: Ok, consertado!!!
# 2022: Já usando o GMX:
# a_spec_name = '../../Genie_Transfer/Filtros/2022/Cci/CCI1603-I.Chn'



In [24]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06322.Chn'

In [25]:
dir(ogra)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'a_spec_ind',
 'a_spec_name',
 'define_files_folder',
 'files_list',
 'home_path',
 'info_mach',
 'info_node',
 'info_plat',
 'info_syst',
 'n_files',
 'pkl_folder_files',
 'process_pickled_list',
 'reduced_f_name',
 'reduced_names_files_list',
 'results_path_name',
 'select_spectrum',
 'spectra_path',
 'spectra_pattern_names']

In [26]:
vars(ogra)

{'info_plat': 'Windows-10-10.0.19045-SP0',
 'info_mach': 'AMD64',
 'info_syst': 'Windows',
 'info_node': 'I56611',
 'home_path': WindowsPath('C:/Users/mmaduar'),
 'spectra_path': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer'),
 'n_files': 7316,
 'a_spec_ind': 0,
 'a_spec_name': '',
 'reduced_f_name': '',
 'files_list': [WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/BGNOVO.Chn'),
  WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/2018/CCI/CCI (I-123) 210218.Chn'),
  WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/2018/CCI/CCI0404F.Chn'),
  WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/2018/CCI/CCI0404I.Chn'),
  WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/2018/CCI/CCI0703F.Chn'),
  WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/2018/CCI/CCI0703I.Chn'),
  WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/2018/CCI/CCI0808I.Chn'),
  WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/2018/CCI/CCI0905 I-123.Chn'),
  WindowsP

In [27]:
# Select a machine/system to read from:
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files
0,Windows-10-10.0.19045-SP0,AMD64,Windows,I56611,C:\Users\mmaduar,C:\Users\mmaduar\Nextcloud\Genie_Transfer,7316,0,,,[C:\Users\mmaduar\Nextcloud\Genie_Transfer\BGN...,"[BGNOVO.Chn, 2018/CCI/CCI (I-123) 210218.Chn, ...","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",WindowsI56611.pkl


In [28]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [29]:
reduc_nms

['BGNOVO.Chn',
 '2018/CCI/CCI (I-123) 210218.Chn',
 '2018/CCI/CCI0404F.Chn',
 '2018/CCI/CCI0404I.Chn',
 '2018/CCI/CCI0703F.Chn',
 '2018/CCI/CCI0703I.Chn',
 '2018/CCI/CCI0808I.Chn',
 '2018/CCI/CCI0905 I-123.Chn',
 '2018/CCI/CCI0905F.Chn',
 '2018/CCI/CCI1104F.Chn',
 '2018/CCI/CCI1104I.Chn',
 '2018/CCI/CCI1403F.Chn',
 '2018/CCI/CCI1403I.Chn',
 '2018/CCI/CCI1508I.Chn',
 '2018/CCI/CCI1605.Chn',
 '2018/CCI/CCI1804F.Chn',
 '2018/CCI/CCI1804I.Chn',
 '2018/CCI/CCI1807.Chn',
 '2018/CCI/CCI2004F-1.Chn',
 '2018/CCI/CCI2004F-2.Chn',
 '2018/CCI/CCI2102F.Chn',
 '2018/CCI/CCI2103F.Chn',
 '2018/CCI/CCI2103I.Chn',
 '2018/CCI/CCI2304F.Chn',
 '2018/CCI/CCI2802F.Chn',
 '2018/CCI/CCI2802I.Chn',
 '2018/CCI/PCI0905 F18.Chn',
 '2018/CCI/PCI1403I.Chn',
 '2018/CRF/CRF0103.Chn',
 '2018/CRF/CRF0111.Chn',
 '2018/CRF/CRF0208.Chn',
 '2018/CRF/CRF0305.Chn',
 '2018/CRF/CRF0410.Chn',
 '2018/CRF/CRF0504.Chn',
 '2018/CRF/CRF0507.Chn',
 '2018/CRF/CRF0609.Chn',
 '2018/CRF/CRF0706.Chn',
 '2018/CRF/CRF0803.Chn',
 '2018/CRF/CR

In [30]:
a_spec_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)

Existing:
Achou! indice=1016, nomearq = almera22/user2023/SI2022/SI06322.Chn
Final choices:
spectra_path: C:\Users\mmaduar\Nextcloud\Genie_Transfer
a_spec_name: C:\Users\mmaduar\Nextcloud\Genie_Transfer\almera22\user2023\SI2022\SI06322.Chn
reduced_f_name: almera22/user2023/SI2022/SI06322.Chn


In [31]:
a_spec_name, reduced_f_name

(WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.Chn'),
 'almera22/user2023/SI2022/SI06322.Chn')

In [32]:
a_spec = Spec(a_spec_name, reduced_f_name)

In [33]:
vars(a_spec)

{'net_spec_ser_an': None,
 'final_composed_baseline': None,
 'f_name': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.Chn'),
 'reduced_f_name': 'almera22/user2023/SI2022/SI06322.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.58,
 'source_datetime': None,
 'results_pkl_file': WindowsPath('.'),
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x1d74630b190>,
 'start_datetime': datetime.datetime(2022, 8, 4, 19, 42, 23),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-063/22 - Agua da Piscina do Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x1d74630b3d0>,
 'nucl_an': <nuclide_analysis_class.NuclideAnalysis at 0x1d74630b5e0>}

In [34]:
# 2023-Jun-13
# PAREI AQUI: levar tudo que está aqui para
# nuclide_identification.ipynb


In [35]:
p_en = a_spec.channel_energy_calib.p_en
p_en

Polynomial([-1.99049842,  0.61104035,  0.        ], domain=[-1,  1], window=[-1,  1], symbol='x')

In [36]:
peaks_df['energy'] = p_en(peaks_df['centroids'])
peaks_df

NameError: name 'peaks_df' is not defined

In [ ]:
# 2023-Mar-21: Identify nuclide candidates in peaks_df from pisciro_eng_df, with energy tolerance en_toler

In [ ]:
big_merge = pd.merge(peaks_df, pisciro_eng_df, how='cross')
big_merge

In [ ]:
en_toler = 2.0

In [ ]:
big_merge['delta_en'] = big_merge.energy - big_merge.Engy
big_merge

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#selection-by-callable
# MaTched ENergies
mt_en = pd.DataFrame(big_merge.loc[lambda df: abs(df['delta_en']) < en_toler, :]) \
[['Engy', 'Nucl', 'rough_sums', 'Yie']]
mt_en
# mt_en[['Engy', 'Nucl']]

In [ ]:

msedg serg set asrhg saertrg

In [ ]:
rng = np.random.default_rng()
# rints = rng.integers(low=0, high=10, size=3)

# 2022-Jun-23: Reativar após consertar o arquivo pickle:
# rints = rng.integers(pkl_df1.count()[0], size=3)
# print(rints)
# type(rints[0])
# some_spec = rints[1]
# pkl_df1.iloc[some_spec]
# a_spec_name = pkl_df1.iloc[some_spec]['FName']
# a_spec_name

In [ ]:
# 2022-Dez-28: Atualizar:
# (2022-Jun-24 Lembrando como estão os defaults de total_analysis(...):
# def total_analysis(self, k_sep_pk=2.0, smoo=3000.0, widths_range=(4.0, 20.0)):)
# smoo=3000 fica razoável, mas faz umas ondas
# smoo=1500 fica horrível, faz uns abismos. Quanto menor, pior
# smoo=400 fica razoável, mas faz umas ondas
# ogra.perform_total_analysis(
#     k_sep_pk=2.0, smoo=4096, widths_range=(4.0, 20.0)
# )

In [ ]:
# 2022-Jun-14
# Verificando o que se tem até agora em a_raw_spec:
# (Pode pular e ir direto aos gráficos)

In [ ]:

# Testes:
# https://numpy.org/doc/stable/reference/random/generator.html#distributions
# x = np.linspace(0, 10, N) + rng.normal(loc=7, scale=0.05, size=N)
# y = np.linspace(22, 23, N)

# Busca do canal onde a altura do espectro se "estabiliza"
# https://numpy.org/doc/stable/reference/routines.sort.html#searching



In [ ]:
vars(pkprms)

In [ ]:
# 2022-Dez-21 Achar o pico de maior área inicial do espectro:

In [ ]:
largest_pk = np.argmax(pkprms.rough_sums)
pkprms.centroids[largest_pk]

In [ ]:
# Só pra lembrar como faz divisão inteira
21 // 7

In [ ]:
wi_re = pkprms.wide_regions[largest_pk]
x = np.linspace(wi_re[0], wi_re[1], num=wi_re[1]-wi_re[0]+1)
y = cntarr[wi_re[0]:wi_re[1]+1]
c1 =  pkprms.centroids[largest_pk]
s1 = pkprms.propts['widths'][largest_pk]*pkprms.k_fwhm_to_sd
a1 = pkprms.rough_sums[largest_pk]
# c2 = 313
# s2 = 1.5
# a2 = 400
sste = 2.0

In [ ]:
# 2023-Jan-3
# Aqui finalizo este teste:
# Dado um espectro, escolhi o pico mais proeminente do espectro
# e fiz o ajuste gaussianco com lmfit
# No caso, o pico tem área, centroide, SD e FWHM:
# g1_amplitude:  5264.84728 +/- 85.1465566 (1.62%) (init = 5347.497)
# g1_center:     1926.29564 +/- 0.02690572 (0.00%) (init = 1926.275)
# g1_sigma:      1.44040103 +/- 0.02690060 (1.87%) (init = 1.418733)
# g1_fwhm:       3.39188516 +/- 0.06334608 (1.87%) == '2.3548200*g1_sigma'
# g1_height:     1458.18438 +/- 23.5824677 (1.62%) == '0.3989423*g1_amplitude/max(1e-15, g1_sigma)'

# Comparando com saíuda do Genie:
# canal (>0)    En:    FWHM (keV)    net area    net area unc   continuum
# 1927.34      1173.47 2.04          5.40E+003   89.73          8.83E+002
#

In [ ]:
out

In [ ]:
out.params

In [ ]:
out.var_names

In [ ]:
out.params.items()

In [ ]:
out.params.pretty_print()

In [ ]:
print('-------------------------------')
print('Parameter    Value       Stderr')
for name, param in out.params.items():
    # print(f'{name:7s} {param.value:11.5f} {param.stderr:11.5f}')
    print(name)
    print(param)
    print(param.value)

In [ ]:
def gauss_lmfit_singlet(pkprms, ipk):
    wi_re = pkprms.wide_regions[ipk]
    print(f'pico: {wi_re}')
    x = np.linspace(wi_re[0], wi_re[1], num=wi_re[1]-wi_re[0]+1)
    y = cntarr[wi_re[0]:wi_re[1]+1]
    c1 = pkprms.centroids[ipk]
    s1 = pkprms.propts['widths'][ipk]*pkprms.k_fwhm_to_sd
    a1 = pkprms.rough_sums[ipk]

    parames = Parameters()

    gauss1 = GaussianModel(prefix='g1_')
    parames.update(gauss1.make_params())
    parames['g1_center'].set(value=c1, min=c1-2, max=c1+2)
    parames['g1_sigma'].set(value=s1, min=0.5, max=10)
    parames['g1_amplitude'].set(value=a1, min=10)

    mod = gauss1

    init = mod.eval(parames, x=x)
    out = mod.fit(y, parames, x=x, verbose=False)
    print('Fez.')

    # ret = [param for param in out.params.items()]
    ret = out
    return ret

In [ ]:
for ipk in range(len(pkprms.peaks)):
    ret = gauss_lmfit_singlet(pkprms, ipk)
    print(ret)

In [ ]:
ret

In [ ]:
ret.params

In [ ]:
for name, param in ret.params.items():
    print(name, param.value, param.stderr)

In [ ]:
for ipk in range(len(pkprms.peaks)):
    ret = gauss_lmfit_singlet(pkprms, ipk)
    # ret.params.pretty_print()

In [ ]:
ret

In [ ]:
xfgn sr hsrthf srth

In [ ]:
my_df.dtypes

In [ ]:
ch_en = ogra.a_spec.channel_energy_calib.p_en

In [ ]:
my_df['centrs_en'] = ch_en(my_df['centroids'])
my_df

In [ ]:
# The End